# 第一回：CHSH不等式

$\newcommand{\ket}[1]{|#1\rangle}$

## とりあえずこのセルを走らせてから授業を始めます

In [ ]:
# まずは必要になるpythonモジュールをすべてインポートしておく
import sys
sys.path.append('/home/jovyan/qc-workbook-lecturenotes/ja')
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, IBMQ, transpile
from qiskit.providers.ibmq import least_busy, IBMQProviderError, IBMQAccountCredentialsNotFound
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram
# qc_workbookはこのワークブック独自のモジュール（インポートエラーが出る場合はPYTHONPATHを設定するか、sys.pathをいじってください）
from qc_workbook.utils import operational_backend

circuits = [QuantumCircuit(2) for _ in range(4)]

for circuit in circuits:
    circuit.h(0)
    circuit.cx(0, 1)
    
circuits[0].ry(-np.pi / 4., 1)

circuits[1].ry(-3. * np.pi / 4., 1)

circuits[2].ry(-np.pi / 4., 1)
circuits[2].ry(-np.pi / 2., 0)

circuits[3].ry(-3. * np.pi / 4., 1)
circuits[3].ry(-np.pi / 2., 0)

for circuit in circuits:
    circuit.measure_all()

IBMQ.load_account()

try:
    provider = IBMQ.get_provider(hub='ibm-q-utokyo', group='internal', project='qc-training22')
except IBMQProviderError:
    provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
    
backend_list = provider.backends(filters=operational_backend(min_qubits=2))

backend = least_busy(backend_list)

shots = backend.configuration().max_shots

circuits = transpile(circuits, backend=backend)
job = backend.run(circuits, shots=shots)

print(f'Provider: {provider.credentials.hub}/{provider.credentials.group}/{provider.credentials.project}')
print(f'Backend: {backend.name()}')
print(f'Job ID: {job.job_id()}')

## 最初の量子回路

In [ ]:
circuit = QuantumCircuit(2) # レジスタを介さずビット数を指定して回路を作成することもできます
circuit.h(0) # その場合、ゲートにはregister[0]ではなく直接量子ビットの番号を指定します
circuit.ry(np.pi / 2., 0) #　θ = π/2
circuit.x(0)
# 実際の回路では出力を得るためには必ず最後に測定を行う
circuit.measure_all()

print(f'This circuit has {circuit.num_qubits} qubits and {circuit.size()} operations')

### 回路を数式で表現

$$
\begin{align}
X R_y\left(\frac{\pi}{2}\right) H \ket{0} & = X R_y\left(\frac{\pi}{2}\right) \frac{1}{\sqrt{2}}(\ket{0} + \ket{1}) \\
& = \frac{1}{\sqrt{2}} X \left[\left(\cos\left(\frac{\pi}{4}\right)\ket{0} + \sin\left(\frac{\pi}{4}\right)\ket{1}\right) + \left(-\sin\left(\frac{\pi}{4}\right)\ket{0} + \cos\left(\frac{\pi}{4}\right)\ket{1}\right)\right] \\
& = \frac{1}{\sqrt{2}} X \frac{1}{\sqrt{2}} \left[\left(\ket{0} + \ket{1}\right) + \left(-\ket{0} + \ket{1}\right)\right] \\
& = X \ket{1} \\
& = \ket{0}
\end{align}
$$

## 制御ゲートを使った例

計算基底$\ket{0}, \ket{1}, \ket{2}, \ket{3}$の振幅の絶対値自乗が$1:2:3:4$の比になるような状態を作ってみましょう。さらに$C^0_1[Z]$ゲートを使って$\ket{3}$だけ振幅の符号が他と異なるようにします。

In [ ]:
theta1 = 2. * np.arctan(np.sqrt(7. / 3.))
theta2 = 2. * np.arctan(np.sqrt(2.))
theta3 = 2. * np.arctan(np.sqrt(4. / 3))

circuit = QuantumCircuit(2)
circuit.ry(theta1, 1)
circuit.ry(theta2, 0)
circuit.cry(theta3 - theta2, 1, 0) # C[Ry]　1が制御で0が標的
circuit.cz(0, 1) # C[Z] 0が制御で1が標的（実はC[Z]ではどちらが制御でも結果は同じ）
circuit.measure_all()

print(f'This circuit has {circuit.num_qubits} qubits and {circuit.size()} operations')

### 回路を数式で表現

$$
\begin{align}
R_y(\theta_1)\ket{0} & = \sqrt{\frac{3}{10}} \ket{0} + \sqrt{\frac{7}{10}} \ket{1} \\
R_y(\theta_2)\ket{0} & = \sqrt{\frac{1}{3}} \ket{0} + \sqrt{\frac{2}{3}} \ket{1} \\
R_y(\theta_3 - \theta_2)R_y(\theta_2)\ket{0} & = R_y(\theta_3)\ket{0} = \sqrt{\frac{3}{7}} \ket{0} + \sqrt{\frac{4}{7}} \ket{1}.
\end{align}
$$

なので、

$$
\begin{align}
& C^1_0[R_y(\theta_3 - \theta_2)]R_{y1}(\theta_1)R_{y0}(\theta_2)\ket{0}_1\ket{0}_0 \\
= & C^1_0[R_y(\theta_3 - \theta_2)]\left(\sqrt{\frac{3}{10}} \ket{0}_1 + \sqrt{\frac{7}{10}} \ket{1}_1\right) R_y(\theta_2)\ket{0}_0\\
= & \sqrt{\frac{3}{10}} \ket{0}_1 R_y(\theta_2)\ket{0}_0 + \sqrt{\frac{7}{10}} \ket{1}_1 R_y(\theta_3)\ket{0}_0 \\
= & \sqrt{\frac{3}{10}} \ket{0}_1 \left(\sqrt{\frac{1}{3}} \ket{0}_0 + \sqrt{\frac{2}{3}} \ket{1}_0\right) + \sqrt{\frac{7}{10}} \ket{1}_1 \left(\sqrt{\frac{3}{7}} \ket{0}_0 + \sqrt{\frac{4}{7}} \ket{1}_0\right) \\
= & \sqrt{\frac{1}{10}} \ket{00} + \sqrt{\frac{2}{10}} \ket{01} + \sqrt{\frac{3}{10}} \ket{10} + \sqrt{\frac{4}{10}} \ket{11} 
\end{align}
$$

最後に$C[Z]$をかけると、$\ket{11}$だけ符号が反転します。

### 回路図の描き方と読み方

In [ ]:
circuit.draw('mpl')

## CHSH不等式を計算する回路を書く

In [ ]:
circuits = []

# 回路I - H, CX[0, 1], Ry(-π/4)[1]をかける
circuit = QuantumCircuit(2, name='circuit_I')
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(-np.pi / 4., 1)
circuit.measure_all()
# 回路リストに追加
circuits.append(circuit)

# 回路II - H, CX[0, 1], Ry(-3π/4)[1]をかける
circuit = QuantumCircuit(2, name='circuit_II')
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(-3. * np.pi / 4., 1)
circuit.measure_all()
# 回路リストに追加
circuits.append(circuit)

# 回路III - H, CX[0, 1], Ry(-π/4)[1], Ry(-π/2)[0]をかける
circuit = QuantumCircuit(2, name='circuit_III')
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(-np.pi / 4., 1)
circuit.ry(-np.pi / 2., 0)
circuit.measure_all()
# 回路リストに追加
circuits.append(circuit)

# 回路IV - H, CX[0, 1], Ry(-3π/4)[1], Ry(-π/2)[0]をかける
circuit = QuantumCircuit(2, name='circuit_IV')
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(-3. * np.pi / 4., 1)
circuit.ry(-np.pi / 2., 0)
circuit.measure_all()
# 回路リストに追加
circuits.append(circuit)

# draw()にmatplotlibのaxesオブジェクトを渡すと、そこに描画してくれる
# 一つのノートブックセルで複数プロットしたい時などに便利
fig, axs = plt.subplots(2, 2, figsize=[12., 6.])
for circuit, ax in zip(circuits, axs.reshape(-1)):
    circuit.draw('mpl', ax=ax)
    ax.set_title(circuit.name)

## 回路を実機で実行する

最初のセルで色々やってしまっているが、改めて実機を使うための手順を紹介。

In [ ]:
# 2回目の認証なのでWarningが出るが気にしない
IBMQ.load_account()

認証が済んだら、利用する量子コンピュータ（「バックエンド」と呼びます）を選びます。

In [ ]:
# IBMQプロバイダ（実機へのアクセスを管理するオブジェクト）
# アカウント登録が済んでいる人は、上のコース専用のプロバイダが使える
try:
    provider = IBMQ.get_provider(hub='ibm-q-utokyo', group='internal', project='qc-training22')
except IBMQProviderError:
    # 済んでいないとエラーになるので、デフォルトのプロバイダを使う
    provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

# バックエンド（実機）のうち量子ビット数2個以上のもののリストをプロバイダから取得する
backend_list = provider.backends(filters=operational_backend(min_qubits=2))

# リストの中から一番空いているものを選ぶ
backend = least_busy(backend_list)

print(f'Jobs will run on {backend.name()}')

回路をトランスパイルしてバックエンドに渡します。

In [ ]:
# バックエンドごとに決められている最大ショット数
shots = backend.configuration().max_shots
print(f'Running four circuits, {shots} shots each')

# transpileの説明は次回の実習にて
circuits = transpile(circuits, backend=backend)
# バックエンドで回路をshots回実行させ、測定結果を返させる
# すでに最初のセルで同じジョブを投げているので、次の行はコメントアウトしてある
# job = backend.run(circuits, shots=shots)

# 最初のセルを走らせた後で何か問題があってjobオブジェクトを再び取り直す必要がある場合
# backend = provider.get_backend('__backend_name__')
# job = backend.retrieve_job('__job_id__')

# ジョブが終了するまで状態を表示しながら待つ（正常に完了、エラーで停止、など終了する理由は一つではない）
job_monitor(job, interval=2)

## 量子測定結果の解析

### ジョブの結果をダウンロードする

In [ ]:
# result()でジョブの結果をIBMのサーバーからダウンロードする
result = job.result()

# 4つの回路のヒストグラムデータを入れるリスト
counts_list = []

# 回路ごとの結果をresultから抽出する
for idx in range(4):
    # get_counts(i)で回路iのヒストグラムデータが得られる
    counts = result.get_counts(idx)
    # データをリストに足す
    counts_list.append(counts)
    
print(counts_list)

### 生データをプロットする

In [ ]:
fig, axs = plt.subplots(2, 2, sharey=True, figsize=[12., 8.])
for counts, circuit, ax in zip(counts_list, circuits, axs.reshape(-1)):
    plot_histogram(counts, ax=ax)
    ax.set_title(circuit.name)
    ax.yaxis.grid(True)

### CHSH不等式に当てはめる

In [ ]:
# C^I, C^II, C^III, C^IVを一つのアレイにする
#（今の場合ただのリストにしてもいいが、純粋な数字の羅列にはnumpy arrayを使うといいことが多い）
C = np.zeros(4, dtype=float)

# enumerate(L)でリストのインデックスと対応する要素に関するループを回せる
for ic, counts in enumerate(counts_list):
    # counts['00'] でなく counts.get('00', 0) - 上のテキストを参照
    C[ic] = counts.get('00', 0) + counts.get('11', 0) - counts.get('01', 0) - counts.get('10', 0)

# 4つの要素を同時にshotsで規格化（リストではこういうことはできない）
C /= shots
    
S = C[0] - C[1] + C[2] + C[3]

print('C:', C)
print('S =', S)
if S > 2.:
    print('Yes, we are using a quantum computer!')
else:
    print('Armonk, we have a problem.')

無事、$S$が2を超えました。

## 参考文献

```{bibliography}
:filter: docname in docnames
```